In [ ]:
import pathlib

import numpy as np
import torch
import torchvision
import tqdm
from PIL import Image, ImageDraw

COLORS = ["blue", "orange", "green", "red"]

In [ ]:
!cd yolov7 && \
    python detect.py \
        --device 1 \
        --weights runs/train/w6-nitti-p6-1856/weights/epoch_174.pt \
        --source ../data/derived/test-ll-1856/ \
        --save-txt \
        --nosave \
        --name w6-nitti-p6-1856-e174 \
        --save-conf \
        --no-trace \
        --augment \
        --iou-thres 1.0 \
        --conf 0.2 \
        --img-size 1856

In [ ]:
with open("output-cropped-test.csv", "w") as fp:
    c = 0
    for path in tqdm.tqdm(sorted(pathlib.Path("./data/original/Norway/test/images").glob("*.jpg"))):
        # Get the name of the image by removing the .jpg extension
        name = path.name[:-4]

        # Get the width and height of the original image
        with Image.open(path) as image:
            width, height = image.width, image.height

        detections_path = pathlib.Path(f"./yolov7/runs/detect/w6-nitti-p6-1856-e174/labels/{name}.txt")
        fp.write(f"{name}.jpg,")
        detections = None
        if detections_path.exists():
            detections = np.loadtxt(detections_path, ndmin=2)

            # Keep only detections above the given confidence threshold
            detections = detections[detections[:, 5] > 0.6]

            # Map the x-center and y-center coordinates relative to the image
            detections[:, 1] *= 1856
            detections[:, 2] = detections[:, 2] * 1856 + (height - 1856)

            # Map width and height
            detections[:, 3] *= 1856 / 2
            detections[:, 4] *= 1856 / 2

            # Get boxes in x1,y1,x2,y2-format from the x_center,y_center,half_width,half_height
            boxes = np.column_stack([
                detections[:, 1] - detections[:, 3],
                detections[:, 2] - detections[:, 4],
                detections[:, 1] + detections[:, 3],
                detections[:, 2] + detections[:, 4],
            ])

            # Find the indices of the top 5 boxes per class using NMS with a given threshold
            indices = []
            for cl in range(4):
                class_indicies = np.argwhere(detections[:, 0] == cl).flatten()
                a = torchvision.ops.nms(
                    torch.from_numpy(boxes[class_indicies]),
                    torch.from_numpy(detections[class_indicies][:, 5]),
                    0.45
                ).numpy()[:5]
                indices.extend(class_indicies[a])

            # Order the remaining indices by decreasing confidence and keep only the top 5
            indices = np.array(indices, dtype=np.int64)[np.argsort(-detections[indices, 5])][:5]

            boxes = boxes.astype(int)
            box_strings = []
            for idx in indices:
                box = boxes[idx]
                box_strings.append(f"{int(detections[idx, 0]) + 1} {box[0]} {box[1]} {box[2]} {box[3]}")
            fp.write(" ".join(box_strings))
        fp.write("\n")

        # with Image.open(pathlib.Path(f"./data/original/Norway/test/images/{name}.jpg")) as image:
        #     if detections is not None:
        #         draw = ImageDraw.Draw(image)
        #         for idx in indices:
        #             xmin, ymin, xmax, ymax = boxes[idx]
        #             draw.rectangle((xmin, ymin, xmax, ymax), fill=None, outline=COLORS[int(detections[idx, 0])], width=4)
        #     image.save(f"out/output-detection-{name}.jpg")
        #     c += 1
        #     if c > 10:
        #         break